# Requerimiento de recursos

## Datos del proyecto en análisis

In [21]:
proyecto="sibayo"
mes_en_analisis=5
anio=2025

In [22]:
mes_recursos_solicitados=mes_en_analisis+1

## Librerias necesarias

In [23]:
import xlsxwriter
import re
import random
import pickle
import pandas as pd
import os
import os
import math
import locale, calendar
import locale
import json
import itertools
import excel2img
import calendar
import pprint
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
from xhtml2pdf import pisa
from typing import List, Dict, Optional
from typing import Dict, List, Any
from pathlib import Path
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
from jinja2 import Template, Environment
from google.oauth2 import service_account
from google.cloud import firestore
from firebase_admin import credentials, firestore
from dotenv import load_dotenv, find_dotenv
from datetime import date, timedelta

In [24]:
locale.setlocale(locale.LC_TIME, 'es_PE.UTF-8')  

'es_PE.UTF-8'

## Funciones útiles

## Carga de datos

### Pickle

### Firestore

In [25]:
## Datos de # 1. Busca el .env en el directorio actual o en cualquiera de los padres
dotenv_path = find_dotenv()
if not dotenv_path:
    raise FileNotFoundError("No se encontró ningún archivo .env en este directorio ni en sus padres.")
load_dotenv(dotenv_path)

# 2. Define el root del proyecto como la carpeta que contiene el .env
project_root = Path(dotenv_path).parent

# 3. Obtén la ruta relativa de las credenciales desde la variable de entorno
rel_cred_path = os.getenv("FIRESTORE_CREDENTIALS")
if not rel_cred_path:
    raise RuntimeError("No existe la variable FIRESTORE_CREDENTIALS en el .env")

# 4. Construye la ruta absoluta al JSON
cred_path = Path(rel_cred_path)
if not cred_path.is_absolute():
    cred_path = (project_root / cred_path).resolve()

if not cred_path.exists():
    raise FileNotFoundError(f"No existe el archivo de credenciales en: {cred_path}")

# 5. Carga las credenciales y crea el cliente de Firestore
credentials = service_account.Credentials.from_service_account_file(str(cred_path))
client = firestore.Client(credentials=credentials, project=credentials.project_id)

# 6. Prueba que funcione
print("Colecciones disponibles:", [c.id for c in client.collections()])

Colecciones disponibles: ['rutinarios']


In [26]:
db=firestore.Client(credentials=credentials, project=credentials.project_id)

#### Desembolsos

In [27]:
desembolsos=db.collection("rutinarios").document(proyecto).collection("presupuestos").document("desembolsos").get().to_dict()

cronograma_desembolsos=desembolsos['cronograma_desembolsos']

pprint.pprint(cronograma_desembolsos)

{'10': {'gastos_operativos': 704.82,
        'igv': 1075.15,
        'mantenimiento_con_go': 7753.01,
        'mantenimiento_con_igv': 7048.19,
        'mantenimiento_sin_igv': 5973.04},
 '11': {'gastos_operativos': 534.7,
        'igv': 815.65,
        'mantenimiento_con_go': 5881.72,
        'mantenimiento_con_igv': 5347.01,
        'mantenimiento_sin_igv': 4531.37},
 '12': {'gastos_operativos': 262.18,
        'igv': 399.93,
        'mantenimiento_con_go': 2883.94,
        'mantenimiento_con_igv': 2621.76,
        'mantenimiento_sin_igv': 2221.83},
 '4': {'gastos_operativos': 222.55,
       'igv': 339.48,
       'mantenimiento_con_go': 2448.06,
       'mantenimiento_con_igv': 2225.51,
       'mantenimiento_sin_igv': 1886.02},
 '5': {'gastos_operativos': 2553.4,
       'igv': 3895.02,
       'mantenimiento_con_go': 28087.43,
       'mantenimiento_con_igv': 25534.03,
       'mantenimiento_sin_igv': 21639.01},
 '6': {'gastos_operativos': 2988.84,
       'igv': 4559.25,
       'mantenim

In [28]:
desembolso_current_month=cronograma_desembolsos[str(mes_recursos_solicitados)]
pprint.pprint(desembolso_current_month)

{'gastos_operativos': 2988.84,
 'igv': 4559.25,
 'mantenimiento_con_go': 32877.25,
 'mantenimiento_con_igv': 29888.41,
 'mantenimiento_sin_igv': 25329.16}


In [29]:
costo_mantenimiento_rutinario=desembolso_current_month['mantenimiento_con_igv']
costo_gastos_operativos=desembolso_current_month['gastos_operativos']
costo_total_mantenimiento=desembolso_current_month['mantenimiento_con_go']
print(f"Costo de mantenimiento rutinario: {costo_mantenimiento_rutinario}")
print(f"Costo de gastos operativos: {costo_gastos_operativos}")
print(f"Costo total de mantenimiento: {costo_total_mantenimiento}")

Costo de mantenimiento rutinario: 29888.41
Costo de gastos operativos: 2988.84
Costo total de mantenimiento: 32877.25


#### Documento firestore

In [30]:
doc_proyecto_firebase = db.collection("rutinarios").document(proyecto).get().to_dict()

#### Datos generales

In [31]:
datos_generales = doc_proyecto_firebase["datos_generales"]

pprint.pprint(datos_generales)

{'distritos': ['Chivay', 'Tuti', 'Sibayo'],
 'provincia': 'Caylloma',
 'region': 'Arequipa'}


In [32]:
distritos, region, provincia = '-'.join(datos_generales['distritos']), datos_generales['region'], datos_generales['provincia']
print(f"Distritos: {distritos}, Región: {region}, Provincia: {provincia}")

Distritos: Chivay-Tuti-Sibayo, Región: Arequipa, Provincia: Caylloma


#### Expediente técnico

In [33]:
expediente = doc_proyecto_firebase["expediente"]

pprint.pprint(expediente)

{'cargas_trabajo': {'MR101': 10.28,
                    'MR103': 53.33,
                    'MR104': 85.33,
                    'MR201': 32350,
                    'MR202': 34.67,
                    'MR301': 15999,
                    'MR401': 64.67,
                    'MR601': 258.02,
                    'VP101': 1512.58,
                    'VP102': 700},
 'codigo_ruta': 'AR-683',
 'coordenadas': {'fin': {'altitud': 3629,
                         'datum': 'WGS84',
                         'hemisferio': 'S',
                         'progresiva': 32252.0,
                         'x': 220383.97,
                         'y': 8270075.15,
                         'zona': None,
                         'zona_letra': None},
                 'inicio': {'altitud': 3820,
                            'datum': 'WGS84',
                            'hemisferio': 'S',
                            'progresiva': 0,
                            'x': 226316.6,
                            'y': 8281146.

#### Código de ruta

In [34]:
codigo_ruta= expediente.get("codigo_ruta", "")
if not codigo_ruta:
    print("No se encontró el código de ruta en los datos del expediente.")
print(f"Código de ruta: {codigo_ruta}")

Código de ruta: AR-683


### Longitud

In [35]:
longitud = expediente.get("longitud", 0)
if longitud is None:
    print("No se encontró la longitud en los datos del expediente.")
print(f"Longitud: {longitud} km")

Longitud: 32252.0 km


In [36]:
contrato=doc_proyecto_firebase["contrato"]
pprint.pprint( contrato)

{'contratista': {'razon_social': 'PLATERS MANAGEMENT S.A.C.', 'ruc': ''},
 'denominacion_tramo_convenio': 'EMP. AR-111 (NUEVO SIBAYO) - TUTI EMP. AR-681 '
                                '(DV. CHIVAY) (KM 32+252)',
 'fecha_inicio': {'anio': 2025, 'dia': 15, 'mes': 4},
 'id_contrato': '004-2025',
 'jefe_mantenimiento': {'apellido': 'Tinta Cáceres',
                        'dni': 0,
                        'nombre': 'Genaro',
                        'titulo': 'Ingeniero'},
 'monto_contrato': 165130.4,
 'numero_cuadrillas': 1,
 'numero_trabajadores': 3,
 'tiempo_ejecucion_dias': 240,
 'tipo_servicio': 'mantenimiento rutinario'}


In [37]:
denominacion_tramo_convenio= contrato["denominacion_tramo_convenio"]
print(denominacion_tramo_convenio)

EMP. AR-111 (NUEVO SIBAYO) - TUTI EMP. AR-681 (DV. CHIVAY) (KM 32+252)


## Cálculos

In [38]:
data_for_document={
  "departamento": region,
  "provincia": provincia,
  "distrito": distritos,
  "tramo": denominacion_tramo_convenio,
  "codigo_ruta": codigo_ruta,
  "categoria": "Camino Vecinal",
  "longitud_km": str(round(longitud/1000, 2)),
  "mes": calendar.month_name[mes_recursos_solicitados],
  "mantenimiento_rutinario": costo_mantenimiento_rutinario,
  "gastos_operativos": costo_gastos_operativos,
  "costo_total_mantenimiento": costo_total_mantenimiento,
}

In [39]:
# —————————————————————————————————————————————
# Filtro para formato de moneda
# —————————————————————————————————————————————
def format_currency(value):
    return f"{value:,.2f}"

# —————————————————————————————————————————————
# Plantilla HTML + CSS (A4 landscape) con tu estructura de tabla
# —————————————————————————————————————————————
html_template = """
<!DOCTYPE html>
<html lang="es">
<head>
  <meta charset="UTF-8"/>
  <style>
    @page { size: A4 landscape; margin: 1cm; }
    body { font-family: sans-serif; font-size: 11pt; }
    .title { text-align: center; font-size: 16pt; font-weight: bold; margin-bottom: 8px; }
    table { width: 100%; border-collapse: collapse; }
    .header-table td { padding: 4px 6px; vertical-align: top; }
    .content-table th {
      background-color: #2C3E50; color: white; padding: 6px; text-align: left; font-size: 10pt;
    }
    .content-table td {
      border: 1px solid #bbb; padding: 6px; vertical-align: top;
      word-break: break-word;
    }
    .content-table tr:nth-child(even) { background-color: #f2f2f2; }
    .footer-row td {
      font-weight: bold; background-color: #dde4ec;
    }
  </style>
</head>
<body>

  <div class="title">Requerimiento de recursos</div>

  <!-- Datos superiores, sin bordes -->
  <table class="header-table">
    <tr>
      <td><strong>CAMINO:</strong> {{ tramo }}</td>
      <td><strong>CATEGORÍA:</strong> {{ categoria }}</td>
      <td><strong>CÓDIGO DE RUTA:</strong> {{ codigo_ruta }}</td>
      <td><strong>LONGITUD TOTAL:</strong> {{ longitud_km }} km</td>
    </tr>
  </table>

  <!-- Tabla principal con rowspan/colspan -->
  <table class="content-table">
    <thead>
      <tr>
        <th rowspan="2">DEPARTAMENTO</th>
        <th rowspan="2">PROVINCIA</th>
        <th rowspan="2">DISTRITO</th>
        <th rowspan="2">TRAMO</th>
        <th colspan="3">{{ mes.upper() }}</th>
      </tr>
      <tr>
        <th>MANTENIMIENTO RUTINARIO</th>
        <th>GASTOS OPERATIVOS</th>
        <th>TOTAL</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>{{ departamento }}</td>
        <td>{{ provincia }}</td>
        <td>{{ distrito }}</td>
        <td>{{ tramo }}</td>
        <td>{{ mantenimiento_rutinario | format_currency }}</td>
        <td>{{ gastos_operativos | format_currency }}</td>
        <td>{{ costo_total_mantenimiento | format_currency }}</td>
      </tr>
      <tr class="footer-row">
        <td colspan="4"><strong>TOTAL</strong></td>
        <td><strong>{{ mantenimiento_rutinario | format_currency }}</strong></td>
        <td><strong>{{ gastos_operativos | format_currency }}</strong></td>
        <td><strong>{{ costo_total_mantenimiento | format_currency }}</strong></td>
      </tr>
    </tbody>
  </table>

</body>
</html>
"""

# —————————————————————————————————————————————
# Crear Environment y registrar filtro
# —————————————————————————————————————————————
env = Environment()
env.filters['format_currency'] = format_currency
template = env.from_string(html_template)

# —————————————————————————————————————————————
# Renderizar HTML y generar PDF con xhtml2pdf
# —————————————————————————————————————————————
html_out = template.render(**data_for_document)

ruta_directorio = os.path.join("output", proyecto, str(mes_en_analisis))
ruta_archivo_requerimiento_recursos = os.path.join(ruta_directorio, "requerimiento_recursos.pdf")

# Crear el directorio si no existe
os.makedirs(ruta_directorio, exist_ok=True)
# Guardar el diccionario en un archivo JSON

def html_to_pdf(source_html: str, output_filename: str):
    with open(output_filename, "wb") as output_file:
        pisa_status = pisa.CreatePDF(source_html, dest=output_file)
    return pisa_status.err

err = html_to_pdf(html_out, ruta_archivo_requerimiento_recursos)
if err:
    print("⚠️ Ocurrió un error al generar el PDF.")
else:
    print("✔️ PDF generado: requerimiento_de_recursos.pdf")

✔️ PDF generado: requerimiento_de_recursos.pdf
